In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


In [ ]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

In [ ]:
df_train.head(10)

One Hot Encoding

In [ ]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [ ]:
one_hot_labels = np.asarray(one_hot)

In [ ]:
im_size = 224

In [ ]:
x_train = []
y_train = []
x_test = []

In [ ]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

In [ ]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

In [ ]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

In [ ]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

In [ ]:
num_class = y_train_raw.shape[1]

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size = 0.3, random_state = 1)

In [ ]:
# Create the base pre-trained model
# Can't download weights in the kernel
base_model = VGG19(#weights='imagenet',
    weights = None, include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'adam', 
              metrics = ['accuracy'])

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3, verbose = 1)
                 ]
model.summary()

In [ ]:
model.fit(X_train, Y_train, epochs = 1, validation_data = (X_valid, Y_valid), verbose = 1)

In [ ]:
preds = model.predict(x_test)

In [ ]:
# Set the highest prediction to 1 and the rest to 0
preds == preds.max(axis = 1, keepdims=True)
preds = (preds == preds.max(axis = 1, keepdims = True)).astype(int)

In [ ]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(5)